Below is a short, step‐by‐step tutorial for building a simple agent that uses LangGraph together with Anthropic’s API to generate LinkedIn post ideas. This example uses Python and assumes you have your Anthropic API key ready. The tutorial is based on the latest documentation and community examples for both LangGraph and Anthropic’s API.

# Tutorial: Building a LinkedIn Post Ideas Generator Agent

## Overview

In this tutorial, you will:

- Set up your Python environment.
- Create a LangGraph that defines a simple state machine.
- Integrate Anthropic’s API (using a Claude model) to generate creative post ideas.
- Run the agent to interactively generate LinkedIn post ideas.

*This example leverages LangGraph to manage the application flow while Anthropic’s LLM (Claude) produces the text.*



## Prerequisites

- **Python 3.8+**
- **API key for Anthropic**:
  - Sign up and obtain your API key from the Anthropic Console [Anthropic Docs](https://docs.anthropic.com/en/api/getting-started)

## Installation of required packages:

Run the following command to install LangGraph and Anthropic’s client:

In [4]:
!pip install -U langgraph anthropic langchain_anthropic


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.5 MB/s eta 0:00:00


# Step 1: Setting Up the Environment

First, set your Anthropic API key as an environment variable to ensure it’s accessible by the Anthropic client.

If your key is stored in a .env or using Jupyter Notebook locally, you can run as follow:

In [ ]:
import os
os.environ["ANTHROPIC_API_KEY"] = "your_anthropic_api_key_here"


If you are using Google Collab - like I did - and stored the key the 'Secrets'tab, run as follow:

In [29]:
from google.colab import userdata
ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
api_key = ANTHROPIC_API_KEY

# Step 2: Creating the LangGraph Agent

Define a simple state machine using LangGraph. In this example, we’ll create a node that calls Anthropic’s ChatAnthropic model (Claude) to generate LinkedIn post ideas.

Here I used the latest Anthropic's model, claude-3-7-sonnet-20250219; but feel free to change it.

Also, to be able to use the API, you need to add some money to your account. More info here: [Anthropic's Website](https://support.anthropic.com/en/articles/8977456-how-do-i-pay-for-my-api-usage)

In [30]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_anthropic import ChatAnthropic

In [31]:
# Define a state schema that holds a list of messages.
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [32]:
# Initialize a LangGraph instance with the state.
graph_builder = StateGraph(State)

# Initialize the Anthropic client (using Claude).
llm = ChatAnthropic(model="claude-3-7-sonnet-20250219", anthropic_api_key=api_key)


In [33]:
# Define the node function that generates post ideas.
def generate_post_ideas(state: State):
    # Take the existing conversation history.
    conversation = state["messages"]
    # Append a query for LinkedIn post ideas.
    query = {"role": "user", "content": "Generate 5 creative LinkedIn post ideas for professional networking."}
    full_prompt = conversation + [query]
    # Get the response from Anthropic.
    response = llm.invoke(full_prompt)
    return {"messages": [response]}

In [34]:
# Add the node to the graph.
graph_builder.add_node("post_idea_generator", generate_post_ideas)
# Define the graph flow.
graph_builder.add_edge(START, "post_idea_generator")
graph_builder.add_edge("post_idea_generator", END)

# Compile the graph into a runnable object.
graph = graph_builder.compile()

## Explanation:

- We create a simple state (holding a list of messages) and a node named "post_idea_generator".
- The node function appends a specific query to generate post ideas and calls Anthropic’s LLM.
- The graph is set up with an entry point and an exit point so that execution stops after the node returns its result.

# Step 3: Running the Agent

Now, create a simple loop to prompt for user input and display the generated LinkedIn post ideas.

In [35]:
def run_agent():
    print("LinkedIn Post Ideas Generator. Type 'quit' to exit.")
    while True:
        user_input = input("Enter some context or leave blank: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Exiting agent. Goodbye!")
            break
        # Initialize the state with the user's input.
        initial_state = {"messages": [{"role": "user", "content": user_input}]}
        # Run the graph and stream the output.
        for event in graph.stream(initial_state):
            for value in event.values():
                print("Agent:", value["messages"][-1].content)

run_agent()


LinkedIn Post Ideas Generator. Type 'quit' to exit.
Enter some context or leave blank: NLP AND PROBABILISTIC MODELS
Agent: # 5 Creative LinkedIn Post Ideas for Professional Networking

## 1. "The NLP Revolution: From Theory to Practice"
Share your journey implementing an NLP model that transformed business outcomes. Include a brief case study with actual results and invite connections to share their own NLP implementation stories. End with a thoughtful question about balancing technical sophistication with practical business value.

## 2. "Probabilistic Models Explained Through Everyday Scenarios"
Create an engaging analogy comparing complex probabilistic concepts to everyday decisions (like choosing lunch options based on past experiences). Include a simple visualization and ask your network how they explain complex statistical concepts to non-technical stakeholders.

## 3. "Weekly NLP Paper Breakdown: What It Means For Industry"
Analyze a recent academic paper in NLP and explain its 

## How it works:

- The agent begins with an initial message (from the user).
- It then adds a fixed query asking for LinkedIn post ideas.
- Anthropic’s API generates a response, which is printed to the console.

# Conclusion

You have now built a simple agent using LangGraph and Anthropic’s API to generate creative LinkedIn post ideas.

This modular design allows you to expand the agent further by:
- Adding more nodes (e.g., for reviewing or editing responses).
- Incorporating memory to maintain multi-turn conversations.
- Integrating additional tools (such as web search) to enhance idea generation.

For further reading:
- [Anthropic API Documentation](https://docs.anthropic.com/en/home)
- [LangGraph Tutorials and Documentation](https://langchain-ai.github.io/langgraph/tutorials/)
Feel free to experiment and customize the agent to suit your specific needs!

---

This completes this short tutorial for building a LinkedIn post ideas generator agent using LangGraph and Anthropic’s API!